# Initialize database tables

## Control table

In [38]:
import os
import numpy as np

from grizli.aws import db
from grizli import utils

import djapipes

In [4]:
djapipes.run.run_pipes_on_dja_spec?

Signature:
djapipes.run.run_pipes_on_dja_spec(
    spec_name,
    sfh='continuity',
    n_age_bins=10,
    scale_disp=1.3,
    dust_type='kriek',
    use_msa_resamp=False,
    fit_agn=False,
    fit_dla=False,
    fit_mlpoly=False,
)
Docstring:
Runs bagpipes on spectrum from DJA AWS database, saves posteriors as .h5 files and plots as .pdf files

Parameters
----------
spec_name : DJA spectrum name, format=str

Returns
-------
None
File:      ~/miniconda3/envs/py310-djapipes/lib/python3.10/site-packages/djapipes/run.py
Type:      function

In [31]:
import time
row = dict(
    file_spec = "rubies-egs61-v3_prism-clear_4233_42328.spec.fits",
    sfh='continuity',
    n_age_bins=10,
    scale_disp=1.3,
    dust_type='kriek',
    use_msa_resamp=False,
    fit_agn=False,
    fit_dla=False,
    fit_mlpoly=False,
    status=70,
    ctime=time.time(),
)

tab = utils.GTable([row])
tab

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,70,1734393587.8455942


In [33]:
if (0):
    # db.execute("DROP TABLE nirspec_bagpipes")
    db.send_to_database("nirspec_bagpipes", tab, if_exists="replace")
    db.execute("CREATE INDEX on nirspec_bagpipes (file_spec)")


db.SQL("select * from nirspec_bagpipes")

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,70,1734393587.8455942


## Results table

In [45]:
# ! ls pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/
post = utils.read_catalog("pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_postcat.csv")
post["sfh"] = "continuity"
post["dust_type"] = "kriek"
for c in post.colnames:
    if post[c].dtype == np.float64:
        post[c] = post[c].astype(np.float32)

if (0):
    db.send_to_database("nirspec_bagpipes_posterior", post, if_exists="replace")
    db.execute("CREATE INDEX on nirspec_bagpipes_posterior (file_spec)")

db.SQL("select * from nirspec_bagpipes_posterior")

file_spec,file_phot,file_zout,id_phot,dr,filt_num,calib_0_16,calib_0_50,calib_0_84,calib_1_16,calib_1_50,calib_1_84,calib_2_16,calib_2_50,calib_2_84,continuity_dsfr1_16,continuity_dsfr1_50,continuity_dsfr1_84,continuity_dsfr2_16,continuity_dsfr2_50,continuity_dsfr2_84,continuity_dsfr3_16,continuity_dsfr3_50,continuity_dsfr3_84,continuity_dsfr4_16,continuity_dsfr4_50,continuity_dsfr4_84,continuity_dsfr5_16,continuity_dsfr5_50,continuity_dsfr5_84,continuity_dsfr6_16,continuity_dsfr6_50,continuity_dsfr6_84,continuity_dsfr7_16,continuity_dsfr7_50,continuity_dsfr7_84,continuity_dsfr8_16,continuity_dsfr8_50,continuity_dsfr8_84,continuity_massformed_16,continuity_massformed_50,continuity_massformed_84,continuity_metallicity_16,continuity_metallicity_50,continuity_metallicity_84,dust_Av_16,dust_Av_50,dust_Av_84,nebular_logU_16,nebular_logU_50,nebular_logU_84,noise_scaling_16,noise_scaling_50,noise_scaling_84,redshift_16,redshift_50,redshift_84,stellar_mass_16,stellar_mass_50,stellar_mass_84,formed_mass_16,formed_mass_50,formed_mass_84,sfr_16,sfr_50,sfr_84,ssfr_16,ssfr_50,ssfr_84,nsfr_16,nsfr_50,nsfr_84,mass_weighted_age_16,mass_weighted_age_50,mass_weighted_age_84,tform_16,tform_50,tform_84,tquench_16,tquench_50,tquench_84,mass_weighted_zmet_16,mass_weighted_zmet_50,mass_weighted_zmet_84,sfh_16,sfh_50,sfh_84,chisq_phot_16,chisq_phot_50,chisq_phot_84,restU_16,restU_50,restU_84,restV_16,restV_50,restV_84,restJ_16,restJ_50,restJ_84,sfh,dust_type
str48,str36,str43,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str10,str5
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,ceers-full-grizli-v7.4-fix_phot.fits,ceers-full-grizli-v7.4-fix.eazypy.zout.fits,45984,0.0025837352,7,1.0906843,1.1007513,1.111982,0.13404463,0.14948104,0.16724166,-0.025392357,-0.01450396,-0.005704524,1.5992963,2.3499546,3.897448,-5.432896,-2.759181,-1.5772108,-2.6653802,-0.920001,0.101954564,-2.2580905,-0.7151477,0.3022016,-1.9315832,-0.53638446,0.43751964,-1.5627255,-0.2930139,0.77155346,-1.3798062,-0.2493546,0.7266544,-1.347817,-0.22493288,0.79783887,10.594977,10.6036625,10.613153,1.9617285,2.0898528,2.2323523,0.26661077,0.31436917,0.3580495,-3.4096978,-2.4284744,-1.4446774,1.6471881,1.7081825,1.7717766,2.6793857,2.6820676,2.6856956,10.358072,10.366877,10.377017,10.594977,10.6036625,10.613153,3.2800497e-08,3.7763068e-05,0.003439571,-17.849339,-14.788182,-12.8288145,-8.708921,-5.647402,-3.6870823,1.2075636,1.2132057,1.2291875,1.158053,1.1736107,1.1806157,1.4938755,1.497463,1.5014544,1.9557912,2.0847487,2.228576,0.0,1.2216088e-05,0.032659084,13.353081,14.23861,15.93904,4.925325,4.945362,4.966345,2.97721,2.9887047,3.0007243,1.7958202,1.8099265,1.8247521,continuity,kriek


# Sync example

In [47]:
sfh = "continuity"
dust_type = "kriek"

file_spec = "rubies-egs61-v3_prism-clear_4233_42328.spec.fits"


In [62]:
import glob

runName = file_spec.split('.spec.fits')[0]
root = file_spec.split('-v3')[0] + '-v3'

s3_path = f"s3://msaexp-nirspec/pipes/{root}"

files = glob.glob(f"pipes/cats/{runName}/*")
files += glob.glob(f"pipes/plots/{runName}/*")
files += glob.glob(f"pipes/posterior/{runName}/*")

for file in files:
    ! aws s3 cp {file} {s3_path}/ --acl public-read


upload: pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_calibcurve.csv to s3://msaexp-nirspec/pipes/rubies-egs61-v3/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_calibcurve.csv
upload: pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_postcat.csv to s3://msaexp-nirspec/pipes/rubies-egs61-v3/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_postcat.csv
upload: pipes/plots/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_calib.png to s3://msaexp-nirspec/pipes/rubies-egs61-v3/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_calib.png
upload: pipes/plots/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_specfit_flam.png to s3://msaexp-nirspec/pipes/rubies-egs61-v3/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_specfit_flam.png
upload: pipes/plots/rubies-egs

# Scripts on EC2

In [76]:
from importlib import reload
reload(djapipes.run); reload(djapipes.utils)
reload(djapipes.run); reload(djapipes.utils)


<module 'djapipes.utils' from '/Users/gbrammer/miniconda3/envs/py310-djapipes/lib/python3.10/site-packages/djapipes/utils.py'>

In [93]:
#!/usr/bin/env python

# Wrapper around djapipes.run

import os
import glob
import yaml
import warnings
import time

import numpy as np
import matplotlib.pyplot as plt
plt.ioff()

import astropy.time

import grizli
from grizli import utils
utils.set_warnings()

HOME = '/GrizliImaging/'
HOME = os.getcwd()

def get_random_bagpipes_file():
    """
    Get a table row of a file to be processed

    Returns
    -------
    res : None, table row
        Returns a single row from `reprocess_rates` with ``status == 0``, or None if
        nothing found

    """
    from grizli.aws import db

    rows = db.SQL(f"""SELECT *
        FROM nirspec_bagpipes
        WHERE status = 0 order by random() limit 1
    """)

    if len(rows) == 0:
        return None
    else:
        return rows


def run_djapipes_wrapper(file_spec="rubies-egs61-v3_prism-clear_4233_42328.spec.fits", sfh="continuity", dust_type="kriek", **kwargs):
    """
    Wrapper around djapipes.run.run_pipes_on_dja_spec
    """
    import glob
    
    import djapipes
    
    status = 0
    
    try:
        djapipes.run.run_pipes_on_dja_spec(
            file_spec=file_spec,
            sfh=sfh,
            dust_type=dust_type,
            **kwargs
        )
        
        status |= 2
    except:
        status |= 4
        return status

    runName = file_spec.split('.spec.fits')[0]
    root = file_spec.split('-v3')[0] + '-v3'

    s3_path = f"s3://msaexp-nirspec/pipes/{root}"

    files = glob.glob(f"pipes/cats/{runName}/*")
    files += glob.glob(f"pipes/plots/{runName}/*")
    files += glob.glob(f"pipes/posterior/{runName}/*")
    
    if len(files) == 0:
        print(f"No outputs found in pipes/*/{runName}")
        
        status |= 8 
    
    for file in files:
        os.system(f"aws s3 cp {file} {s3_path}/ --acl public-read")
    
    # Posterior result
    post_file = f"pipes/cats/{runName}/{runName}_{sfh}_{dust_type}_postcat.csv"
    if os.path.exists(post_file):
        
        print(f'Send {post_file} to nirspec_bagpipes_posterior')
        
        post = utils.read_catalog(post_file)
        post["sfh"] = sfh
        post["dust_type"] = dust_type
    
        for c in post.colnames:
            if post[c].dtype == np.float64:
                post[c] = post[c].astype(np.float32)
        
        # Remove row if it currently exists
        db.execute(f"""
        DELETE FROM nirspec_bagpipes_posterior
        WHERE file_spec = '{file_spec}'
        AND sfh = '{sfh}'
        AND dust_type = '{dust_type}'
        """)
        
        db.send_to_database("nirspec_bagpipes_posterior", post, if_exists="append")
    else:
        print("posterior summary {post_file} not found")
        status |= 16
    
    return status

    
def run_one():
    """
    Run a file with status = 0
    """
    import os
    import time
    import boto3

    from grizli.aws import db
    from grizli import utils


    row = get_random_bagpipes_file()

    if row is None:
        print('Nothing to do.')
        with open(os.path.join(HOME, 'nirspec_pipes_finished.txt'),'w') as fp:
            fp.write(time.ctime() + '\n')

        return None

    print(f'============  NIRSpec dja-pipes  ==============')
    print(f"{row['file_spec'][0]}")
    print(f'========= {time.ctime()} ==========')

    file_prefix = row['file_spec'][0].split('.spec')[0]
    # key = row['root'][0] + '-' + file_prefix
    key = f"{file_prefix}-pipes"

    WORK_PATH = os.path.join(HOME, key)
    if os.path.exists(WORK_PATH):
        print('! already exists, skip')
        return row
    
    ORIG_PATH = os.getcwd()
    
    os.makedirs(WORK_PATH)    
    os.chdir(WORK_PATH)
    
    with open(os.path.join(HOME, 'nirspec_pipes_history.txt'),'a') as fp:
        fp.write(f"{time.ctime()} {row['file_spec'][0]}\n")

    # Update db
    db.execute(f"""
        UPDATE nirspec_bagpipes
        SET status = 1, ctime={time.time()}
        WHERE
            file_spec = '{row['file_spec'][0]}'
        """
    )

    #################
    # Run it
    #################
    status = run_djapipes_wrapper(**row[0])

    # Update db
    db.execute(f"""
        UPDATE nirspec_bagpipes
        SET
            status = {status},
            ctime = {time.time()}
        WHERE
            file_spec = '{row['file_spec'][0]}'
        """
    )

    os.chdir(ORIG_PATH)

    return row


# if __name__ == "__main__":
#     run_one()




In [84]:
run_djapipes_wrapper()

42328 rubies-egs61-v3_prism-clear_4233_42328
100% [............................................................................] 639360 / 639360
Results loaded from pipes/posterior/rubies-egs61-v3_prism-clear_4233_42328/42328.h5

Fitting not performed as results have already been loaded from pipes/posterior/rubies-egs61-v3_prism-clear_4233_42328/42328.h5. To start over delete this file or change run.

upload: pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_calibcurve.csv to s3://msaexp-nirspec/pipes/rubies-egs61-v3/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_calibcurve.csv
upload: pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_postcat.csv to s3://msaexp-nirspec/pipes/rubies-egs61-v3/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_postcat.csv
upload: pipes/plots/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kr

2

In [86]:
rows = db.SQL("select * from nirspec_bagpipes")
rows

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,70,1734393587.8455942


In [90]:
# New row
rows = rows[:1]
rows['file_spec'] = "rubies-uds43-v3_prism-clear_4233_31823.spec.fits"
rows['status'] = 0
db.send_to_database("nirspec_bagpipes", rows, if_exists='append')

1

In [94]:
db.SQL("select * from nirspec_bagpipes")

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,70,1734393587.8455942
rubies-uds43-v3_prism-clear_4233_31823.spec.fits,continuity,10,1.3,kriek,False,False,False,False,0,1734393587.8455942


In [95]:
row = run_one()

============  NIRSpec dja-pipes  ==============
rubies-uds43-v3_prism-clear_4233_31823.spec.fits
========= Tue Dec 17 02:05:53 2024 ==========
31823 rubies-uds43-v3_prism-clear_4233_31823
100% [............................................................................] 642240 / 642240
Bagpipes: fitting object 31823


Completed in 355.6 seconds.

Parameter                          Posterior percentiles
                                16th       50th       84th
----------------------------------------------------------
calib:0                        1.276      1.288      1.300
calib:1                       -0.025     -0.007      0.009
calib:2                       -0.017     -0.009     -0.000
continuity:dsfr1               1.583      2.313      4.074
continuity:dsfr2              -6.489     -3.834     -2.348
continuity:dsfr3              -2.194     -0.725      0.223
continuity:dsfr4              -2.081     -0.623      0.387
continuity:dsfr5              -1.845     -0.497      0.478
con

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "WHERE"
LINE 6:         WHERE
                ^

[SQL: 
        UPDATE nirspec_bagpipes
        SET
            status = 2,
            ctime = 1734397969.853627,
        WHERE
            file_spec = 'rubies-uds43-v3_prism-clear_4233_31823.spec.fits'
        ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [98]:
# Status changed to 2
db.SQL("select * from nirspec_bagpipes")

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,70,1734393587.8455942
rubies-uds43-v3_prism-clear_4233_31823.spec.fits,continuity,10,1.3,kriek,False,False,False,False,2,1734397969.853627


In [119]:
if 0:
    db.execute("""ALTER TABLE nirspec_bagpipes_posterior
ADD djapipes_version VARCHAR(64);""")


In [120]:
# Results are in the posterior table
db.SQL("select * from nirspec_bagpipes_posterior")

file_spec,file_phot,file_zout,id_phot,dr,filt_num,calib_0_16,calib_0_50,calib_0_84,calib_1_16,calib_1_50,calib_1_84,calib_2_16,calib_2_50,calib_2_84,continuity_dsfr1_16,continuity_dsfr1_50,continuity_dsfr1_84,continuity_dsfr2_16,continuity_dsfr2_50,continuity_dsfr2_84,continuity_dsfr3_16,continuity_dsfr3_50,continuity_dsfr3_84,continuity_dsfr4_16,continuity_dsfr4_50,continuity_dsfr4_84,continuity_dsfr5_16,continuity_dsfr5_50,continuity_dsfr5_84,continuity_dsfr6_16,continuity_dsfr6_50,continuity_dsfr6_84,continuity_dsfr7_16,continuity_dsfr7_50,continuity_dsfr7_84,continuity_dsfr8_16,continuity_dsfr8_50,continuity_dsfr8_84,continuity_massformed_16,continuity_massformed_50,continuity_massformed_84,continuity_metallicity_16,continuity_metallicity_50,continuity_metallicity_84,dust_Av_16,dust_Av_50,dust_Av_84,nebular_logU_16,nebular_logU_50,nebular_logU_84,noise_scaling_16,noise_scaling_50,noise_scaling_84,redshift_16,redshift_50,redshift_84,stellar_mass_16,stellar_mass_50,stellar_mass_84,formed_mass_16,formed_mass_50,formed_mass_84,sfr_16,sfr_50,sfr_84,ssfr_16,ssfr_50,ssfr_84,nsfr_16,nsfr_50,nsfr_84,mass_weighted_age_16,mass_weighted_age_50,mass_weighted_age_84,tform_16,tform_50,tform_84,tquench_16,tquench_50,tquench_84,mass_weighted_zmet_16,mass_weighted_zmet_50,mass_weighted_zmet_84,sfh_16,sfh_50,sfh_84,chisq_phot_16,chisq_phot_50,chisq_phot_84,restU_16,restU_50,restU_84,restV_16,restV_50,restV_84,restJ_16,restJ_50,restJ_84,sfh,dust_type,djapipes_version
str48,str42,str49,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str10,str5,object
rubies-uds43-v3_prism-clear_4233_31823.spec.fits,primer-uds-south-grizli-v7.2-fix_phot.fits,primer-uds-south-grizli-v7.2-fix.eazypy.zout.fits,31823,1.0683849e-08,7,1.2750163,1.2873616,1.3006295,-0.024277417,-0.0069819586,0.008741275,-0.016894482,-0.009070048,-5.0981787e-05,1.53419,2.2769182,4.101171,-6.347813,-3.683133,-2.164358,-2.155918,-0.7631303,0.20011736,-2.0920408,-0.5782702,0.3504977,-1.7171866,-0.4967887,0.526128,-1.7692782,-0.411543,0.6400344,-1.7593238,-0.30391493,0.7695858,-1.2967153,-0.08319812,0.9829348,11.100544,11.108019,11.114423,1.823329,1.918836,2.0125654,0.500041,0.53605044,0.5629376,-3.5010028,-2.5242841,-1.4703611,1.9587781,2.0237625,2.0929391,2.6001453,2.6020393,2.6041963,10.86431,10.872096,10.878323,11.100544,11.108019,11.114423,2.7328499e-08,4.2361982e-05,0.0053142607,-18.440039,-15.242527,-13.151957,-9.286204,-6.087121,-3.9949243,1.2118473,1.2161918,1.2402385,1.2269682,1.2498184,1.2558264,1.5733256,1.5755416,1.5774307,1.8170892,1.912741,2.0069196,0.0,1.3401974e-05,0.047588248,10.566063,12.500886,15.903134,3.8637443,3.8844194,3.905697,1.8094065,1.8210313,1.832904,0.50435734,0.51576585,0.52664465,continuity,kriek,--
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,ceers-full-grizli-v7.4-fix_phot.fits,ceers-full-grizli-v7.4-fix.eazypy.zout.fits,45984,0.0025837352,7,1.0892755,1.1018765,1.1126999,0.13211712,0.15095365,0.16584659,-0.023812044,-0.014497551,-0.0033406895,1.6189506,2.4097276,3.7875385,-5.6111374,-2.7355287,-1.4944911,-2.5760698,-0.92896944,0.05555089,-1.9344946,-0.66059303,0.309189,-2.1085901,-0.5317176,0.41920167,-1.8360478,-0.4230325,0.61577797,-1.7018257,-0.28566298,0.67321867,-1.4699566,-0.253742,0.73665625,10.594298,10.603997,10.612552,1.9681497,2.0981338,2

In [122]:
if (1):
    db.execute("update nirspec_bagpipes set status = 0 where file_spec = 'rubies-egs61-v3_prism-clear_4233_42328.spec.fits'")

db.SQL("select * from nirspec_bagpipes")

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-uds43-v3_prism-clear_4233_31823.spec.fits,continuity,10,1.3,kriek,False,False,False,False,2,1734397969.853627
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,0,1734428003.8802009


## Preliminary RUBIES sample

In [123]:
catalog_list = djapipes.utils.preferred_catalogs()
latest_catalogs = ','.join([f"'{c}'" for c in catalog_list['file_phot']])
latest_catalogs

"'abell2744clu-grizli-v7.2-fix_phot.fits','abell2744par-grizli-v7.0-fix_phot.fits','abell370-full-grizli-v7.1-fix_phot.fits','abells1063-grizli-v7.0-fix_phot.fits','ceers-full-grizli-v7.4-fix_phot.fits','ceers-full-grizli-v7.2-fix_phot.fits','elgordo-grizli-v7.0-fix_phot.fits','gdn-grizli-v7.3-fix_phot.fits','gds-grizli-v7.2-fix_phot.fits','gds-sw-grizli-v7.0-fix_phot.fits','ngdeep-grizli-v7.2-fix_phot.fits','j112716p4228-grizli-v7.0-fix_phot.fits','macs0416-clu-grizli-v7.0-fix_phot.fits','macs0417-full-grizli-v7.2-fix_phot.fits','macs0647-grizli-v7.0-fix_phot.fits','macs1149-clu-grizli-v7.0-fix_phot.fits','macs1423-full-grizli-v7.2-fix_phot.fits','mrg0138-grizli-v7.2-fix_phot.fits','primer-cosmos-east-grizli-v7.0-fix_phot.fits','primer-cosmos-west-grizli-v7.0-fix_phot.fits','primer-uds-north-grizli-v7.2-fix_phot.fits','primer-uds-south-grizli-v7.2-fix_phot.fits','rxcj0600-grizli-v7.2-fix_phot.fits','rxj2129-grizli-v7.0-fix_phot.fits','smacs0723-grizli-v7.0-fix_phot.fits','sunrise-griz

In [124]:
# Specify "best" version of a particular photometric catalog
manual_latest = """
abell2744clu-grizli-v7.2-fix_phot.fits
abell2744par-grizli-v7.0-fix_phot.fits
abell370-full-grizli-v7.1-fix_phot.fits
abells1063-grizli-v7.0-fix_phot.fits
ceers-full-grizli-v7.4-fix_phot.fits
#ceers-full-grizli-v7.2-fix_phot.fits
elgordo-grizli-v7.0-fix_phot.fits
gdn-grizli-v7.3-fix_phot.fits
gds-grizli-v7.2-fix_phot.fits
gds-sw-grizli-v7.0-fix_phot.fits
ngdeep-grizli-v7.2-fix_phot.fits
j112716p4228-grizli-v7.0-fix_phot.fits
macs0416-clu-grizli-v7.0-fix_phot.fits
macs0417-full-grizli-v7.2-fix_phot.fits
macs0647-grizli-v7.0-fix_phot.fits
macs1149-clu-grizli-v7.0-fix_phot.fits
macs1423-full-grizli-v7.2-fix_phot.fits
mrg0138-grizli-v7.2-fix_phot.fits
primer-cosmos-east-grizli-v7.0-fix_phot.fits
primer-cosmos-west-grizli-v7.0-fix_phot.fits
primer-uds-north-grizli-v7.2-fix_phot.fits
primer-uds-south-grizli-v7.2-fix_phot.fits
rxcj0600-grizli-v7.2-fix_phot.fits
rxj2129-grizli-v7.0-fix_phot.fits
smacs0723-grizli-v7.0-fix_phot.fits
sunrise-grizli-v7.0-fix_phot.fits
ulasj1342-grizli-v7.2-fix_phot.fits
"""

latest = []
for f in manual_latest.split('\n'):
    if (len(f.strip()) == 0) | (f.strip().startswith('#')):
        continue
    latest.append(f)
        
latest_catalogs = ','.join([f"'{l}'" for l in latest])

### Big query
rubies_mat = db.SQL(f"""
SELECT ne.grating, ne.filter, ne.file as exposure_file, ne.root, nr.z, nrm.z as z_manual, nrm.grade,
       gr.*,
       nr.line_ha_nii, nr.eqw_ha_nii, nr.line_oiii, nr.line_oiii_5007
FROM nirspec_extractions ne,
     nirspec_redshifts nr,
     (SELECT nrm.file, nrm.z, nrm.grade          /* most recent grade */
             FROM nirspec_redshifts_manual nrm,
                  (SELECT nrm2.file, MAX(nrm2.ctime) as max_ctime
                         FROM nirspec_redshifts_manual nrm2
                         GROUP BY nrm2.file
                  ) nrm_sorted
             WHERE nrm.ctime >= nrm_sorted.max_ctime
             AND nrm.file = nrm_sorted.file
             AND nrm.z > 0
     ) nrm,
     nirspec_phot_match nrp,
     grizli_photometry gr
WHERE (ne.root LIKE '%%v3')
AND (ne.root LIKE '%%-nod-v3')
AND (ne.grating LIKE 'PRISM')
AND (ne.root LIKE 'rubies%%')
AND ne.file = nr.file
AND nrm.grade = 3
AND nr.z > 3 AND nr.z < 6
AND nrp.dr < 0.2 AND nrp.file_phot in ({latest_catalogs})
AND (nrp.file_phot = gr.file_phot AND nrp.id_phot = gr.id_phot) /* match to photometry */
AND ne.file = nrp.file_spec
AND ne.file = nrm.file
ORDER BY grating
""")

In [128]:
rubies_mat

grating,filter,exposure_file,root,z,z_manual,grade,file_phot,file_zout,id_phot,thresh,npix,tnpix,xmin_image,xmax_image,ymin_image,ymax_image,x2_image,y2_image,xy_image,errx2,erry2,errxy,a_image,b_image,theta_image,cxx_image,cyy_image,cxy_image,cflux,flux,cpeak,peak,xcpeak,ycpeak,xpeak,ypeak,x_image,y_image,ra_phot,dec_phot,flux_iso,fluxerr_iso,area_iso,mag_iso,kron_radius,kron_rcirc,flux_auto,fluxerr_auto,flux_radius_20,flux_radius,flux_radius_90,area_auto,mag_auto,magerr_auto,f435w_tot_0,f435w_etot_0,f435w_mask_aper_0,f435w_flux_aper_0,f435w_fluxerr_aper_0,f435w_tot_1,f435w_etot_1,f435w_mask_aper_1,f435w_flux_aper_1,f435w_fluxerr_aper_1,f435w_tot_2,f435w_etot_2,f435w_mask_aper_2,f435w_flux_aper_2,f435w_fluxerr_aper_2,f606w_tot_0,f606w_etot_0,f606w_mask_aper_0,f606w_flux_aper_0,f606w_fluxerr_aper_0,f606w_tot_1,f606w_etot_1,f606w_mask_aper_1,f606w_flux_aper_1,f606w_fluxerr_aper_1,f606w_tot_2,f606w_etot_2,f606w_mask_aper_2,f606w_flux_aper_2,f606w_fluxerr_aper_2,f814w_tot_0,f814w_etot_0,f814w_mask_aper_0,f814w_flux_aper_0,f814w_fluxerr_aper_0,f814w_tot_1,f814w_etot_1,f814w_mask_aper_1,f814w_flux_aper_1,f814w_fluxerr_aper_1,f814w_tot_2,f814w_etot_2,f814w_mask_aper_2,f814w_flux_aper_2,f814w_fluxerr_aper_2,f105w_tot_0,f105w_etot_0,f105w_mask_aper_0,f105w_flux_aper_0,f105w_fluxerr_aper_0,f105w_tot_1,f105w_etot_1,f105w_mask_aper_1,f105w_flux_aper_1,f105w_fluxerr_aper_1,f105w_tot_2,f105w_etot_2,f105w_mask_aper_2,f105w_flux_aper_2,f105w_fluxerr_aper_2,f125w_tot_0,f125w_etot_0,f125w_mask_aper_0,f125w_flux_aper_0,f125w_fluxerr_aper_0,f125w_tot_1,f125w_etot_1,f125w_mask_aper_1,f125w_flux_aper_1,f125w_fluxerr_aper_1,f125w_tot_2,f125w_etot_2,f125w_mask_aper_2,f125w_flux_aper_2,f125w_fluxerr_aper_2,f140w_tot_0,f140w_etot_0,f140w_mask_aper_0,f140w_flux_aper_0,f140w_fluxerr_aper_0,f140w_tot_1,f140w_etot_1,f140w_mask_aper_1,f140w_flux_aper_1,f140w_fluxerr_aper_1,f140w_tot_2,f140w_etot_2,f140w_mask_aper_2,f140w_flux_aper_2,f140w_fluxerr_aper_2,f160w_tot_0,f160w_etot_0,f160w_mask_aper_0,f160w_flux_aper_0,f160w_fluxerr_aper_0,f160w_tot_1,f160w_etot_1,f160w_mask_aper_1,f160w_flux_aper_1,f160w_fluxerr_aper_1,f160w_tot_2,f160w_etot_2,f160w_mask_aper_2,f160w_flux_aper_2,f160w_fluxerr_aper_2,f090w_tot_0,f090w_etot_0,f090w_mask_aper_0,f090w_flux_aper_0,f090w_fluxerr_aper_0,f090w_tot_1,f090w_etot_1,f090w_mask_aper_1,f090w_flux_aper_1,f090w_fluxerr_aper_1,f090w_tot_2,f090w_etot_2,f090w_mask_aper_2,f090w_flux_aper_2,f090w_fluxerr_aper_2,f115w_tot_0,f115w_etot_0,f115w_mask_aper_0,f115w_flux_aper_0,f115w_fluxerr_aper_0,f115w_tot_1,f115w_etot_1,f115w_mask_aper_1,f115w_flux_aper_1,f115w_fluxerr_aper_1,f115w_tot_2,f115w_etot_2,f115w_mask_aper_2,f115w_flux_aper_2,f115w_fluxerr_aper_2,f150w_tot_0,f150w_etot_0,f150w_mask_aper_0,f150w_flux_aper_0,f150w_fluxerr_aper_0,f150w_tot_1,f150w_etot_1,f150w_mask_aper_1,f150w_flux_aper_1,f150w_fluxerr_aper_1,f150w_tot_2,f150w_etot_2,f150w_mask_aper_2,f150w_flux_aper_2,f150w_fluxerr_aper_2,f200w_tot_0,f200w_etot_0,f200w_mask_aper_0,f200w_flux_aper_0,f200w_fluxerr_aper_0,f200w_tot_1,f200w_etot_1,f200w_mask_aper_1,f200w_flux_aper_1,f200w_fluxerr_aper_1,f200w_tot_2,f200w_etot_2,f200w_mask_aper_2,f200w_flux_aper_2,f200w_fluxerr_aper_2,f277w_tot_0,f277w_etot_0,f277w_mask_aper_0,f277w_flux_aper_0,f277w_fluxerr_aper_0,f277w_tot_1,f277w_etot_1,f277w_mask_aper_1,f277w_flux_aper_1,f277w_fluxerr_aper_1,f277w_tot_2,f277w_etot_2,f277w_mask_aper_2,f277w_flux_aper_2,f277w_fluxerr_aper_2,f356w_tot_0,f356w_etot_0,f356w_mask_aper_0,f356w_flux_aper_0,f356w_fluxerr_aper_0,f356w_tot_1,f356w_etot_1,f356w_mask_aper_1,f356w_flux_aper_1,f356w_fluxerr_aper_1,f356w_tot_2,f356w_etot_2,f356w_mask_aper_2,f356w_flux_aper_2,f356w_fluxerr_aper_2,f410m_tot_0,f410m_etot_0,f410m_mask_aper_0,f410m_flux_aper_0,f410m_fluxerr_aper_0,f410m_tot_1,f410m_etot_1,f410m_mask_aper_1,f410m_flux_aper_1,f410m_fluxerr_aper_1,f410m_tot_2,f410m_etot_2,f410m_mask_aper_2,f410m_flux_aper_2,f410m_fluxerr_aper_2,f444w_tot_0,f444w_etot_0,f444w_mask_aper_0,f444w_flux_aper_0,f444w_fluxerr

In [127]:
utils.Unique(rubies_mat['root'])

   N  value     
====  ==========
  29  rubies-egs51-nod-v3
  42  rubies-uds31-nod-v3
  47  rubies-egs52-nod-v3
  51  rubies-egs61-nod-v3
  51  rubies-uds43-nod-v3
  52  rubies-uds32-nod-v3
  56  rubies-uds3-nod-v3
  57  rubies-uds1-nod-v3
  60  rubies-uds42-nod-v3
  61  rubies-uds33-nod-v3
  67  rubies-egs53-nod-v3
  67  rubies-uds2-nod-v3
  70  rubies-uds41-nod-v3
  83  rubies-egs63-nod-v3
  87  rubies-egs62-nod-v3


In [137]:
rows = db.SQL("select * from nirspec_bagpipes")
rubies_files = np.unique(rubies_mat['exposure_file'])

to_db = utils.GTable()
to_db['file_spec'] = rubies_files
to_db['status'] = 70

for c in rows.colnames:
    if c in ['ctime']:
        continue
        
    if c not in to_db.colnames:
        print(c)
        to_db[c] = rows[c][0]

already_in_db = np.isin(to_db['file_spec'], rows['file_spec'])
already_in_db.sum()

sfh
n_age_bins
scale_disp
dust_type
use_msa_resamp
fit_agn
fit_dla
fit_mlpoly


0

In [138]:
# db.send_to_database("nirspec_bagpipes", to_db[~already_in_db], if_exists="append")

860

In [144]:
db.SQL("select * from nirspec_bagpipes_posterior")

file_spec,file_phot,file_zout,id_phot,dr,filt_num,calib_0_16,calib_0_50,calib_0_84,calib_1_16,calib_1_50,calib_1_84,calib_2_16,calib_2_50,calib_2_84,continuity_dsfr1_16,continuity_dsfr1_50,continuity_dsfr1_84,continuity_dsfr2_16,continuity_dsfr2_50,continuity_dsfr2_84,continuity_dsfr3_16,continuity_dsfr3_50,continuity_dsfr3_84,continuity_dsfr4_16,continuity_dsfr4_50,continuity_dsfr4_84,continuity_dsfr5_16,continuity_dsfr5_50,continuity_dsfr5_84,continuity_dsfr6_16,continuity_dsfr6_50,continuity_dsfr6_84,continuity_dsfr7_16,continuity_dsfr7_50,continuity_dsfr7_84,continuity_dsfr8_16,continuity_dsfr8_50,continuity_dsfr8_84,continuity_massformed_16,continuity_massformed_50,continuity_massformed_84,continuity_metallicity_16,continuity_metallicity_50,continuity_metallicity_84,dust_Av_16,dust_Av_50,dust_Av_84,nebular_logU_16,nebular_logU_50,nebular_logU_84,noise_scaling_16,noise_scaling_50,noise_scaling_84,redshift_16,redshift_50,redshift_84,stellar_mass_16,stellar_mass_50,stellar_mass_84,formed_mass_16,formed_mass_50,formed_mass_84,sfr_16,sfr_50,sfr_84,ssfr_16,ssfr_50,ssfr_84,nsfr_16,nsfr_50,nsfr_84,mass_weighted_age_16,mass_weighted_age_50,mass_weighted_age_84,tform_16,tform_50,tform_84,tquench_16,tquench_50,tquench_84,mass_weighted_zmet_16,mass_weighted_zmet_50,mass_weighted_zmet_84,sfh_16,sfh_50,sfh_84,chisq_phot_16,chisq_phot_50,chisq_phot_84,restU_16,restU_50,restU_84,restV_16,restV_50,restV_84,restJ_16,restJ_50,restJ_84,sfh,dust_type,djapipes_version
str48,str42,str49,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str10,str5,object
rubies-uds43-v3_prism-clear_4233_31823.spec.fits,primer-uds-south-grizli-v7.2-fix_phot.fits,primer-uds-south-grizli-v7.2-fix.eazypy.zout.fits,31823,1.0683849e-08,7,1.2750163,1.2873616,1.3006295,-0.024277417,-0.0069819586,0.008741275,-0.016894482,-0.009070048,-5.0981787e-05,1.53419,2.2769182,4.101171,-6.347813,-3.683133,-2.164358,-2.155918,-0.7631303,0.20011736,-2.0920408,-0.5782702,0.3504977,-1.7171866,-0.4967887,0.526128,-1.7692782,-0.411543,0.6400344,-1.7593238,-0.30391493,0.7695858,-1.2967153,-0.08319812,0.9829348,11.100544,11.108019,11.114423,1.823329,1.918836,2.0125654,0.500041,0.53605044,0.5629376,-3.5010028,-2.5242841,-1.4703611,1.9587781,2.0237625,2.0929391,2.6001453,2.6020393,2.6041963,10.86431,10.872096,10.878323,11.100544,11.108019,11.114423,2.7328499e-08,4.2361982e-05,0.0053142607,-18.440039,-15.242527,-13.151957,-9.286204,-6.087121,-3.9949243,1.2118473,1.2161918,1.2402385,1.2269682,1.2498184,1.2558264,1.5733256,1.5755416,1.5774307,1.8170892,1.912741,2.0069196,0.0,1.3401974e-05,0.047588248,10.566063,12.500886,15.903134,3.8637443,3.8844194,3.905697,1.8094065,1.8210313,1.832904,0.50435734,0.51576585,0.52664465,continuity,kriek,--
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,ceers-full-grizli-v7.4-fix_phot.fits,ceers-full-grizli-v7.4-fix.eazypy.zout.fits,45984,0.0025837352,7,1.0889778,1.0997196,1.1128951,0.13088687,0.14782424,0.16586438,-0.024130184,-0.0147497365,-0.0046085007,1.6957691,2.4800065,4.160615,-5.626228,-2.7383296,-1.4897332,-2.5928242,-0.9734439,0.008915976,-2.0853574,-0.6415909,0.25972807,-1.9237522,-0.5941174,0.42570898,-1.7983714,-0.50556076,0.45647213,-1.5960901,-0.2714883,0.53949237,-1.3596076,-0.14995532,0.8997528,10.593526,10.603152,10.6112795,1.9675092,2.1098475

In [143]:
db.SQL("select * from nirspec_bagpipes")

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,2,1734430739.0452485
rubies-uds43-v3_prism-clear_4233_31823.spec.fits,continuity,10,1.3,kriek,False,False,False,False,2,1734397969.853627


In [146]:
files = db.SQL("""select file_spec
from nirspec_bagpipes where status = 70
""")
roots = [file.split('-v3')[0] + '-v3' for file in files['file_spec']]
utils.Unique(roots)

   N  value     
====  ==========
  29  rubies-egs51-nod-v3
  42  rubies-uds31-nod-v3
  47  rubies-egs52-nod-v3
  47  rubies-uds3-nod-v3
  51  rubies-egs61-nod-v3
  51  rubies-uds43-nod-v3
  52  rubies-uds32-nod-v3
  56  rubies-uds2-nod-v3
  57  rubies-uds1-nod-v3
  60  rubies-uds42-nod-v3
  61  rubies-uds33-nod-v3
  67  rubies-egs53-nod-v3
  70  rubies-uds41-nod-v3
  83  rubies-egs63-nod-v3
  87  rubies-egs62-nod-v3


In [147]:
# Run one subset
if (0):
    db.execute("update nirspec_bagpipes set status = 0 and status = 70 where file_spec like 'rubies-egs51-nod-v3%%'")

# Launch instances

In [148]:
count = db.SQL("""select count(*) from nirspec_bagpipes where status = 0""")['count'][0]
print(f"status=0  N: {count}")

files = db.SQL("""select file_spec
from nirspec_bagpipes where status = 0
""")
roots = [file.split('-v3')[0] + '-v3' for file in files['file_spec']]
utils.Unique(roots)

status=0  N: 29
   N  value     
====  ==========
  29  rubies-egs51-nod-v3


In [149]:
import os

def launch_ec2_instances(nmax=50, count=None, filter_tiles=True, templ='lt-0e8c2b8611c9029eb,Version=43'):
    """
    Launch EC2 instances from a launch template that run through all 
    status=0 associations/tiles and then terminate
    
    Version 19 is the latest run_all_visits.sh
    Version 20 is the latest run_all_tiles.sh
    Version 24 is run_all_visits with a new python39 environment
    
    Version 25 is run_all_tile_filters.sh with a new python39 environment
    Version 31 is run_all_tile_filters.sh with a new python39 environment and ImageMagick requirement
    Version 32 is run_all_tile_filters.sh with a new python39 environment and ImageMagick requirement - fix bug
    
    Version 34 is run_all_tiles copied from 32
    Version 35 is run_all_tiles copied from 32, with golfir needed for catalog

    Version 41 is run_all_nirspec_preprocess.sh for NIRSpec preprocessing
    Version 42 is run_all_nirspec_preprocess.sh for NIRSpec preprocessing with msaexp.git@update-psf-lookup

    Version 43 is run_all_nirspec_preprocess.sh for NIRSpec preprocessing with msaexp.git@update-psf-lookup
    """

    if count == 0:
        print('No associations to run, abort.')
        return True
    else:
        print(f'# Launch {count} instances with LaunchTemplateId={templ}: ')
        cmd = f'aws ec2 run-instances --count {count}'
        cmd += f' --launch-template LaunchTemplateId={templ}'
        print(cmd)
        
        if templ is not None:
            os.system(cmd)

In [151]:
from grizli.aws import db, visit_processor
if (0) & (count > 0):
    nmax = 30
    
    nlaunch = int(np.minimum(nmax, count))
    print(nlaunch)

    _ = launch_ec2_instances(nmax=nmax, count=nlaunch, templ='lt-0e8c2b8611c9029eb,Version=43')
else:
    print("Don't launch instances")

Don't launch instances


## Check status counts

In [152]:
db.SQL("""select status, count(status) from nirspec_bagpipes group by status""")

status,count
int64,int64
0,29
70,831
2,2


In [162]:
db.SQL("""select status, count(status) from nirspec_bagpipes group by status""")

status,count
int64,int64
1,25
70,831
32,4
2,2
